In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
df_brand = pd.read_csv("./use_data/expanded_training.csv")
df_brand

,Unnamed: 0,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,label
0,31,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
1,32,379824,198,656,145,3462.0,1111,0,5.0,1.0,0
2,33,379824,198,656,145,3462.0,1111,2,5.0,1.0,0
3,34,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
4,35,379824,198,656,145,3462.0,1110,0,5.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
380,6463,122632,175,1181,4760,247.0,1109,0,3.0,0.0,0
381,6464,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
382,6465,122632,175,1181,4760,247.0,1108,0,3.0,0.0,0
383,6508,95362,253,962,3263,626.0,1111,0,0.0,0.0,0
